In [ ]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [ ]:
# reshape the data
X_train = X_train.reshape((X_train.shape[0], 28, 28))
X_test = X_test.reshape((X_test.shape[0], 28, 28))

# change the type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# convert data to 3 channels
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train) # one-hot encoding
y_test = to_categorical(y_test)

In [ ]:
X_train.shape, X_test.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [ ]:
# data augmentation with generator
train_generator = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 40, # augmention of images to avoid overfitting
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow(X_train, y_train, batch_size=512, shuffle=True)

val_iterator = val_generator.flow(X_test, y_test, batch_size=512, shuffle=False)

**ResNet50**

In [ ]:
# Create the Model

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
# add the pretrained model
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
# add fully connected layer with output
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

# set resnet layers not trainable
model.layers[0].trainable=False
model.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24641930 (94.00 MB)
Trainable params: 1054218 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_iterator, epochs=5, validation_data=val_iterator)

Epoch 1/5
118/118 [==============================] - 38s 300ms/step - loss: 1.4439 - accuracy: 0.5579 - val_loss: 0.7813 - val_accuracy: 0.7820
Epoch 2/5
118/118 [==============================] - 34s 287ms/step - loss: 0.8708 - accuracy: 0.7358 - val_loss: 0.6178 - val_accuracy: 0.8083
Epoch 3/5
118/118 [==============================] - 34s 288ms/step - loss: 0.7076 - accuracy: 0.7854 - val_loss: 0.5326 - val_accuracy: 0.8327
Epoch 4/5
118/118 [==============================] - 34s 288ms/step - loss: 0.6118 - accuracy: 0.8131 - val_loss: 0.4599 - val_accuracy: 0.8590
Epoch 5/5
118/118 [==============================] - 34s 287ms/step - loss: 0.5606 - accuracy: 0.8269 - val_loss: 0.4427 - val_accuracy: 0.8611


In [ ]:
# Evaluating the model

test_generator = ImageDataGenerator(rescale = 1./255)

test_iterator = test_generator.flow(X_test, y_test, batch_size=512, shuffle=False)

test_loss, test_acc = model.evaluate(test_iterator) #batch size = 32
print('Test accuracy:', test_acc)

20/20 [==============================] - 3s 169ms/step - loss: 0.4427 - accuracy: 0.8611
Test accuracy: 0.8611000180244446


**VGG16**

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape the data
X_train = X_train.reshape((X_train.shape[0], 28, 28))
X_test = X_test.reshape((X_test.shape[0], 28, 28))

# change the type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# convert data to 3 channels
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train) # one-hot encoding
y_test = to_categorical(y_test)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Resize the images 48*48 as required by VGG16

from keras.preprocessing.image import img_to_array, array_to_img

X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_test])

X_train.shape, X_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

In [ ]:
input_layer=layers.Input(shape=(48,48,3))

# initialize the transfer model VGG16 with appropriate properties per our need.
# we are passing paramers as following
# 1) weights='imagenet' - Using this we are carring weights as of original weights.
# 2) input_tensor to pass the VGG16 using input_tensor
# 3) we want to change the last layer so we are not including top layer
model_vgg16=VGG16(weights='imagenet',input_tensor=input_layer,include_top=False)

last_layer=model_vgg16.output # we are taking last layer of the model

# Add flatten layer: we are extending Neural Network by adding flattn layer
flatten=layers.Flatten()(last_layer)

# Add dense layer
dense1=layers.Dense(100,activation='relu')(flatten)
dense1=layers.Dense(100,activation='relu')(flatten)
dense1=layers.Dense(100,activation='relu')(flatten)


# Add dense layer to the final output layer
output_layer=layers.Dense(10,activation='softmax')(flatten)

# Creating model with input and output layer
model=models.Model(inputs=input_layer,outputs=output_layer)

# Summarize the model
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                              

In [ ]:
print("We are making all the layers intrainable except the last layer. \n")
for layer in model.layers[:-1]:
    layer.trainable=False
model.summary()

We are making all the layers intrainable except the last layer. 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                             

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=128, verbose=True, validation_data=(X_test,y_test))

Epoch 1/2
469/469 [==============================] - 83s 176ms/step - loss: 1.1877 - accuracy: 0.7714 - val_loss: 0.3303 - val_accuracy: 0.8979
Epoch 2/2
469/469 [==============================] - 84s 178ms/step - loss: 0.2704 - accuracy: 0.9165 - val_loss: 0.2292 - val_accuracy: 0.9295


In [ ]:
# Evaluating the model

test_loss, test_acc = model.evaluate(X_test,y_test) #batch size = 32
print('Test accuracy:', test_acc)

313/313 [==============================] - 19s 60ms/step - loss: 0.2292 - accuracy: 0.9295
Test accuracy: 0.9294999837875366


**InceptionV3**

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape the data
X_train = X_train.reshape((X_train.shape[0], 28, 28))
X_test = X_test.reshape((X_test.shape[0], 28, 28))

# change the type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# convert data to 3 channels
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train) # one-hot encoding
y_test = to_categorical(y_test)

In [ ]:
# Resize the images

from keras.preprocessing.image import img_to_array, array_to_img

X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in X_test])

X_train.shape, X_test.shape

((60000, 150, 150, 3), (10000, 150, 150, 3))

In [ ]:
# data augmentation with generator
train_generator = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 40, # augmention of images to avoid overfitting
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow(X_train, y_train, batch_size=512, shuffle=True)

val_iterator = val_generator.flow(X_test, y_test, batch_size=512, shuffle=False)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [ ]:
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import models

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(10, activation='softmax')(x)

model = models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(train_iterator, validation_data = val_iterator, epochs = 2)


Epoch 1/2
118/118 [==============================] - 378s 3s/step - loss: 0.4828 - accuracy: 0.8515 - val_loss: 0.1440 - val_accuracy: 0.9576
Epoch 2/2
118/118 [==============================] - 376s 3s/step - loss: 0.2244 - accuracy: 0.9286 - val_loss: 0.1382 - val_accuracy: 0.9588


In [ ]:
# Evaluating the model

test_generator = ImageDataGenerator(rescale = 1./255)

test_iterator = test_generator.flow(X_test, y_test, batch_size=512, shuffle=False)

test_loss, test_acc = model.evaluate(test_iterator) #batch size = 32
print('Test accuracy:', test_acc)

20/20 [==============================] - 25s 1s/step - loss: 0.1382 - accuracy: 0.9588
Test accuracy: 0.9588000178337097
